# Import Libraries

In [1]:
import pandas as pd
import numpy as np
import re
import os

import torch
import torch.nn as nn
from torch.utils.data import Dataset
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.transforms as transforms

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import median_absolute_error
from sklearn.metrics import r2_score

import wandb
wandb.login()

wandb: Currently logged in as: oe_sarmanova (use `wandb login --relogin` to force relogin)


True

# Dataset for MLP

In [2]:
class OA_Dataset(Dataset):
    def __init__(self, y_file, data_file, transform=None, target_transform=None):
        self.spec_labels = pd.read_csv(y_file, sep=',').iloc[:,1:]
        self.spec_number = pd.read_csv(y_file, sep=',').iloc[:,0]
        self.spec = pd.read_csv(data_file, sep=',').iloc[:,1:]
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.spec_labels)

    def __getitem__(self, idx):
        label = self.spec_labels.iloc[idx]
        number =  self.spec_number.iloc[idx]
        sp = np.array(self.spec.iloc[(self.spec_number.iloc[idx]-1),:], dtype='float32')
        spec = torch.from_numpy(sp)
        if self.transform:
            spec = self.transform(spec)
        if self.target_transform:
            label = self.target_transform(label)
        return number, spec, torch.from_numpy(np.array(label, dtype='float32'))

# Utilities for feature processing

In [3]:
class MinMax_Normalize:
    def __init__(self,feat_min,feat_max):
        assert feat_min.shape == feat_max.shape
        self.min = feat_min
        self.max = feat_max

    def __call__(self,x):
        # x of shape [batch_size, features]
        #assert x.shape[1] == self.min.shape[0]
        return (x - self.min) / (self.max - self.min)

In [4]:
#Calculate min and max
def min_max(train_dataloader):
    data = next(iter(train_dataloader))
    MIN = torch.min(data[1],0).values
    MAX = torch.max(data[1],0).values

    return MIN, MAX

# MLP Architecture

In [5]:
# MLP_1W approach
class FCNN(nn.Module):

    def __init__(self,in_num,latent_num,out_num):
      
        super().__init__() # since Python 3.0
        self.in_num = in_num
        self.latent_num = latent_num
        self.out_num = out_num

        self.layers = nn.Sequential(
            nn.Linear(self.in_num,self.latent_num),
            nn.Sigmoid(),
            nn.Linear(self.latent_num,self.out_num)
        )

    def forward(self, x):
        return self.layers(x)

In [6]:
#1D CNN class (basic)
class OneDCNN(nn.Module):

    def __init__(self):
      
        super().__init__() # since Python 3.0

        self.layers = nn.Sequential(
            nn.Conv1d(in_channels=1, out_channels=2, kernel_size=9),
            nn.Sigmoid(),
            nn.MaxPool1d(2),
            nn.Conv1d(in_channels=2, out_channels=4, kernel_size=9),
            nn.Sigmoid(),
            nn.MaxPool1d(2),
            nn.Flatten(),
            nn.Linear(461*4, 64),
            nn.Sigmoid(),
            nn.Linear(64, 1)
        )
    
    def forward(self, x):
        return self.layers(x)

# Utilities for output

In [7]:
#Write the outputs of the autoencoder
def write_predictions(N, ion, model_name, split_path, val_loss, dataloader, feat_num, dset):
    
    checkpoint = torch.load(split_path+'model'+model_name+'_vld_loss_'+str(int(val_loss))+'.pth')
    N.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    last_best_epoch = checkpoint['epoch']
    loss = checkpoint['loss']
    N.eval()
    out, true = torch.zeros(1, feat_num+1), torch.zeros(1, feat_num+1)
    
    for num, specs, labels in dataloader:
        outputs = N(specs)
        outputs[outputs<0]=0
        
        out = torch.cat((out, torch.cat((num.unsqueeze(-1), outputs), -1)),0)
        true = torch.cat((true, torch.cat((num.unsqueeze(-1), labels), -1)),0)

    a = ['sample_number',ion]
    
    pd.DataFrame(out.detach().numpy()).to_csv(split_path + 'Y_out_'+dset+'.csv',sep=',', header = a)
    pd.DataFrame(true.detach().numpy()).to_csv(split_path + 'Y_true_'+dset+'.csv',sep=',', header = a)

In [8]:
def clear_folder(folder, model_name, loss_value):
    for filename in os.listdir(folder):
        pat = r'.+\.pth'
        if re.match(pat, filename):
            letters = 15+len(model_name)
            if int(filename[letters:-4])!=int(loss_value):os.remove(folder+filename)
            else:
                continue

In [9]:
def reset_weights(m):
    '''
    Reset model weights.
    '''
    for layer in m.children():
        if hasattr(layer, 'reset_parameters'):
            print(f'Reset trainable parameters of layer = {layer}')
            layer.reset_parameters()

# Hyper parameters

In [10]:
input_num = 1868
outs_num = 1
neurons_num = 128 # neurons in  hidden layer

k = 2
ion_name = 'sugar_gL'

batch_size = 64
trn_batch = 1203
vld_batch = 343
tst_batch = 173

learning_rate = 0.001
l2_lambda = 0.001
test_stop = 100         #stopping criterion is an error on the validation set
                        #we stop if the error on the validation set (val_loss) did not fall within 250 epochs (test_stop)
INITIALS = 5 # number of weight's initializations

k_folds = [[42,12],[612,45],[72,172],[871,48],[52,134],[139,15],[287,403],[32,29]]


# Paths

In [11]:
gen_path = 'C:\\Users\\helga\\OneDrive\\Рабочий стол\\Lab\\2024\\wines\\' # path to the directory containing data and results of future calculations

path_to_data = gen_path + 'data\\wines_ir_cleared_spec.csv' # spectra
Y_data = gen_path + 'data\\wines_ir_cleared_conc.csv' #Y DATA

case_name = 'wines_ir_' + ion_name + '_MLP_N01_128_lr_'+str(learning_rate)+'_l2_'+str(l2_lambda)+'_stop_'+str(test_stop)
case_path = gen_path + case_name+'\\'

model_name = '_MLP_N01_128'

In [12]:
Y = pd.read_csv(Y_data, sep=',') # import data with ions concentrations
ion = Y.iloc[:,[0,k]]

# Initialize model

In [13]:
model = FCNN(input_num,neurons_num,outs_num)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
loss_function = torch.nn.MSELoss()

# Learning

In [14]:
for fold in k_folds:
    
    split_path = case_path+'split_'+ str(fold[0])+'_'+str(fold[1])+ '/'
    os.makedirs(split_path, exist_ok=True)

    Y_trn, Y_30 = train_test_split(ion, test_size=0.3, random_state=fold[0])
    Y_vld, Y_tst = train_test_split(Y_30, test_size = 0.3333, random_state=fold[1])
    
    a = ['sample_number',ion_name]

    pd.DataFrame(Y_trn).to_csv(split_path + 'Y_trn.csv',sep=',', index=False, header = a)
    pd.DataFrame(Y_vld).to_csv(split_path + 'Y_vld.csv',sep=',', index=False, header = a)
    pd.DataFrame(Y_tst).to_csv(split_path + 'Y_tst.csv',sep=',', index=False, header = a) 
    
    split_path = case_path+'split_'+ str(fold[0])+'_'+str(fold[1])+ '\\'
    
    #Data import and normalization
    training_data =  OA_Dataset(split_path+'Y_trn.csv',path_to_data)
    train_dataloader = DataLoader(training_data, batch_size=batch_size, shuffle=True)
    
    min_f, max_f = min_max(train_dataloader)

    training_data =  OA_Dataset(split_path+'Y_trn.csv', path_to_data, transform= transforms.Compose([MinMax_Normalize(min_f, max_f)]))
    validation_data =  OA_Dataset(split_path+'Y_vld.csv', path_to_data, transform= transforms.Compose([MinMax_Normalize(min_f, max_f)]))
    test_data =  OA_Dataset(split_path+'Y_tst.csv', path_to_data, transform= transforms.Compose([MinMax_Normalize(min_f, max_f)]))

    train_dataloader = DataLoader(training_data, batch_size=batch_size, shuffle=True)
    validation_dataloader = DataLoader(validation_data, batch_size=vld_batch, shuffle=True)
    test_dataloader = DataLoader(test_data, batch_size=tst_batch, shuffle=True)

    for init_number in range(0,INITIALS): # multiple initialization of network weights
        
        init_path = split_path + str(init_number)+'\\'
        os.makedirs(init_path, exist_ok=True)

        min_val_loss = [1000000000000000.0] #just big random number to enter learning cycle

        wandb.init(project = case_name)

        split_name = 'split_'+ str(fold[0])+'_'+str(fold[1])
        init_name = '_' + str(init_number)
        wandb.run.name = split_name + init_name + model_name
        wandb.run.save()

        for epoch_step in range(0, 5000, test_stop):
            best_epoch=0
            
            if epoch_step!=0:
                
                checkpoint = torch.load(init_path + 'model'+model_name+'_vld_loss_'+str(int(min_val_loss[0]))+'.pth')
                model.load_state_dict(checkpoint['model_state_dict'])
                optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
                last_best_epoch = checkpoint['epoch']
                loss = checkpoint['loss']
                model.train()
                
                if last_best_epoch + test_stop > ep:
                    for ep in range(epoch_step, epoch_step+test_stop):
                        for _, data in enumerate(train_dataloader, 0): # get bacth
                            num, inputs, labels = data # parse batch
                            optimizer.zero_grad() # sets the gradients of all optimized tensors to zero.
                            outputs = model(inputs) # get outputs
                            loss = loss_function(outputs, labels) # calculate loss
                            #Regularization
                            l2_norm = sum(p.pow(2.0).sum()
                                          for p in model.parameters())
                            loss = loss + l2_lambda * l2_norm
                            loss.backward() # calculate gradients
                            optimizer.step() # performs a single optimization step (parameter update).

                        val_loss = 0.0
                        for num, specs, labels in validation_dataloader: 
                            outs = model(specs)
                            val_loss += loss_function(outs, labels)
                        
                        wandb.log({"trn_loss": loss, "vld_loss": val_loss})
                        if val_loss.item() <= min(min_val_loss):
                            torch.save({'epoch': ep,
                              'model_state_dict': model.state_dict(),
                              'optimizer_state_dict': optimizer.state_dict(),
                              'loss': loss}, init_path + 'model'+ model_name + '_vld_loss_' + str(int(round(val_loss.item(),3))) + '.pth')
                            min_val_loss.append(round(val_loss.item(),3))
                        #print(ep, loss, val_loss)
                    clear_folder(init_path,model_name,min(min_val_loss))
                    min_val_loss = [min(min_val_loss)]
                else: continue    
      
            if epoch_step==0:
                model.apply(reset_weights)
                for ep in range(epoch_step, test_stop):
                    for _, data in enumerate(train_dataloader, 0): # get bacth
                        num, inputs, labels = data # parse batch
                        optimizer.zero_grad() # sets the gradients of all optimized tensors to zero.
                        outputs = model(inputs) # get outputs
                        loss = loss_function(outputs, labels)
                        #Regularization
                        l2_norm = sum(p.pow(2.0).sum()
                                      for p in model.parameters())
                        loss = loss + l2_lambda * l2_norm
                        loss.backward() # calculate gradients
                        optimizer.step() # performs a single optimization step (parameter update).

                    val_loss = 0.0
                    for num, specs, labels in validation_dataloader:
                        outs = model(specs)
                        val_loss += loss_function(outs, labels)

                    wandb.log({"trn_loss": loss, "vld_loss": val_loss})
                    if val_loss.item() <= min(min_val_loss):
                        torch.save({'epoch': ep,
                          'model_state_dict': model.state_dict(),
                          'optimizer_state_dict': optimizer.state_dict(),
                          'loss': loss}, init_path + 'model'+model_name+'_vld_loss_'+str(int(round(val_loss.item(),3)))+'.pth')
                        min_val_loss.append(round(val_loss.item(),3))
                    #print(ep, loss, val_loss)
                clear_folder(init_path,model_name,min(min_val_loss))
                min_val_loss = [min(min_val_loss)]


        wandb.log({"epoch": last_best_epoch})
        wandb.finish()
        
        write_predictions(model, ion_name, model_name, init_path, min_val_loss[0], train_dataloader, outs_num, dset = 'trn')
        write_predictions(model, ion_name, model_name, init_path, min_val_loss[0], validation_dataloader, outs_num, dset = 'vld')
        write_predictions(model, ion_name, model_name, init_path, min_val_loss[0], test_dataloader, outs_num, dset = 'tst')

wandb: wandb version 0.17.0 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


wandb: WARNING Calling run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


Reset trainable parameters of layer = Linear(in_features=1868, out_features=128, bias=True)
Reset trainable parameters of layer = Linear(in_features=128, out_features=1, bias=True)


trn_loss,14.72557
vld_loss,2299.88867
_step,600
_runtime,307
_timestamp,1715170098
epoch,495


trn_loss,█▇▄▃▃▂▃▂▂▁▂▂▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
vld_loss,█▇▂▅▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
_runtime,▁▁▁▁▂▂▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇███
_timestamp,▁▁▁▁▂▂▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇███
epoch,▁


...

wandb: wandb version 0.17.0 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


wandb: WARNING Calling run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


Reset trainable parameters of layer = Linear(in_features=1868, out_features=128, bias=True)
Reset trainable parameters of layer = Linear(in_features=128, out_features=1, bias=True)


trn_loss,590.98633
vld_loss,156.07742
_step,500
_runtime,254
_timestamp,1715181124
epoch,391


trn_loss,█▇▃▃▄▂▂▂▁▁▁▁▂▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
vld_loss,█▂▁▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_runtime,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_timestamp,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch,▁
